In [1]:
%pip install deap

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from deap import base, tools, creator, algorithms
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
import pandas as pd

2024-01-14 18:01:26.768779: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# California Housing Dataset'ini yükleme ve eğitim/test setlerine ayırma
X, y = fetch_california_housing(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


In [4]:

# Veri normalizasyonu
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [5]:
# Yapay sinir ağı modelini oluşturma fonksiyonu
def create_model(num_neurons, dropout_rate, learning_rate):
    model = Sequential()
    model.add(Dense(num_neurons, input_dim=X_train.shape[1], activation='relu'))  # Giriş katmanı
    model.add(Dropout(dropout_rate))  # Dropout katmanı
    model.add(Dense(1, activation='linear'))  # Çıkış katmanı
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)  # Optimizer tanımı
    model.compile(loss='mean_squared_error', optimizer=optimizer)  # Modeli derleme
    return model

In [6]:
# Genetik algoritma için fitness fonksiyonu
def eval_nn(individual):
    num_neurons, dropout_rate, learning_rate = individual
    model = create_model(int(num_neurons), dropout_rate, learning_rate)
    early_stopping = EarlyStopping(monitor='val_loss', patience=5)  # Erken durdurma
    model.fit(X_train, y_train, epochs=100, verbose=0, validation_split=0.1, callbacks=[early_stopping])
    loss = model.evaluate(X_test, y_test, verbose=0)
    return (loss,)

In [7]:
# GA için temel yapıların oluşturulması
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("attr_int", np.random.randint, 10, 100)  # Nöron sayısı için rastgele değer
toolbox.register("attr_float", np.random.uniform, 0.0, 1.0)  # Dropout oranı ve learning rate için rastgele değer
toolbox.register("individual", tools.initRepeat, creator.Individual, lambda: [toolbox.attr_int(), toolbox.attr_float(), toolbox.attr_float()], n=3)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", eval_nn)
toolbox.register("mate", tools.cxTwoPoint)  # Çaprazlama fonksiyonu
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.1)  # Mutasyon fonksiyonu
toolbox.register("select", tools.selTournament, tournsize=3)  # Seçim fonksiyonu


In [8]:
# GA'nın çalıştırılması ve her nesil için en iyi bireylerin kaydedilmesi
population = toolbox.population(n=500)
ngen = 10
history = []

In [9]:
for gen in range(ngen):
    # Popülasyonda çaprazlama ve mutasyon işlemi yapılır
    population = algorithms.varAnd(population, toolbox, cxpb=0.5, mutpb=0.2)
    # Bireylerin fitness değerleri hesaplanır
    fits = toolbox.map(toolbox.evaluate, population)
    for fit, ind in zip(fits, population):
        ind.fitness.values = fit
    # Yeni nesil için bireyler seçilir
    population = toolbox.select(population, k=len(population))

    # Nesil içinde en iyi birey bulunur ve kaydedilir
    top_individual = tools.selBest(population, k=1)[0]
    history.append([gen, top_individual, top_individual.fitness.values[0]])


TypeError: 'float' object is not iterable

In [ ]:
# Sonuçların DataFrame olarak yazdırılması
df = pd.DataFrame(history, columns=['Generation', 'Individual', 'Fitness'])
print(df)

Empty DataFrame
Columns: [Generation, Individual, Fitness]
Index: []
